# Modeling Strategies for Songwriter Index

In [90]:
import json
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import StandardScaler 

Determining strategies to use in feature selection & engineering when building the songwriter recommender.

Current thought about ideating through the dataset is looking at each "chunk" of information that I have, and seeing specifically how I can engineer each portion

## Incorporating Audio Analysis

The audio analysis endpoint is made up of an assortment of distinct datasets within a single object:

|Key|Value Type|Value Description|
|---|---|---|
|bars|an array of time interval objects|The time intervals of the bars throughout the track. A bar (or measure) is a segment of time defined as a given number of beats. Bar offsets also indicate downbeats, the first beat of the measure|
|beats|an array of time interval objects|The time intervals of beats throughout the track. A beat is the basic time unit of a piece of music; for example, each tick of a metronome. Beats are typically multiples of tatums.|
|sections|an array of section objects|Sections are defined by large variations in rhythm or timbre, e.g. chorus, verse, bridge, guitar solo, etc. Each section contains its own descriptions of tempo, key, mode, time_signature, and loudness.|
segments|an array of segment objects|Audio segments attempts to subdivide a song into many segments, with each segment containing a roughly consitent sound throughout its duration.|
tatums|an array of time interval objects|A tatum represents the lowest regular pulse train that a listener intuitively infers from the timing of perceived musical events (segments). For more information about tatums, see Rhythm (below).|

So essentially, we're looking at **time interval objects**, **section objects**, and **segment objects**.

### Potential Ideas

#### SVD

Performing SVD on segment objects per song, perhaps limiting each to 50 components. 

Grabbing the Mean and Variance of tatum duration per song

Afterwards, performing SVD again on each of the songs tatum and segment objects tied to a single writer (no weighting applied for writing percentage). And then capturing the other dimensions with some other sort of aggregation:

- Genres?
- Key?

With each of the below, perhaps I can just use the mean and variance of all songs tied to a particular writer:

- Tempo (perhaps mean & variance?)
- Mode (perhaps just mean)
- Time Signature?
- Danceability?
- Energy?
- Valence?
- Speechiness?
- Acousticness?

##### Pros
- Seems like it would capture the necessary latent factors that make a song a song.

##### Cons
- Less interpretable, then say, computing the mean and variance of those values

#### Testing SVD

In [4]:
with open('/Users/jonjohnson/../../Volumes/gdrive/SWI_data/data/new_tracks_20190103.json', 'r') as f:
    tracks = json.load(f)

In [6]:
len(tracks)

651517

In [8]:
tracks_to_sample = random.sample(tracks.keys(), 10)

In [9]:
tracks_to_sample

['4RQMYkHQrEiwvbKtlaW2w8',
 '3cjQIgkhmxnahbeoDmiyFe',
 '58jyyd5QoWnT9Cc3q3ozLG',
 '4naeJLeZDIHxV8hsvU220v',
 '5H2j7EbqkqvbOBwacvo0Vu',
 '4TXGv208zoONRBIkDtzp68',
 '1EkjxpyMHq6iP6ASzrbCpi',
 '4QWrHRsIaMixg0YD9yfMAw',
 '2IziixGyxzwsjX54CdfrNk',
 '5RuotSKQoklW1NER0fuOCn']

In [11]:
def retreive_json(lst):
    track_dict = {}
    for i in lst:
        with open(f'/Users/jonjohnson/../../Volumes/gdrive/SWI_data/data/audio_analysis/{i}.json', 'r') as f:
            track = json.load(f)
        track_dict[i] = track
    return track_dict

In [12]:
tracks_dict = retreive_json(tracks_to_sample)

In [65]:
np.hsplit(np.array(tracks_dict['4RQMYkHQrEiwvbKtlaW2w8']['segments'][0]['pitches']), 1)

[array([0.29 , 0.137, 0.159, 0.166, 0.229, 0.478, 0.461, 0.081, 0.148,
        0.371, 1.   , 0.926])]

##### Extracting Pitch and Timbre Values for each Song 

In [83]:
def pitch_timbre_grabber(dct):
    pitches = np.hsplit(np.array(dct['segments'][0]['pitches']), 1)
    timbre = np.hsplit(np.array(dct['segments'][0]['pitches']), 1)
    for i in range(1, len(dct['segments'])):
        pitches = np.hstack([pitches, np.hsplit(np.array(dct['segments'][i]['pitches']), 1)])
        timbre = np.hstack([timbre, np.hsplit(np.array(dct['segments'][i]['timbre']), 1)])
    return pitches, timbre

In [84]:
p,t = pitch_timbre_grabber(tracks_dict['4RQMYkHQrEiwvbKtlaW2w8'])

In [91]:
pca = PCA(50, random_state=333)

Pitch vector is transposed below, because the number of components must be less than the number of features (columns).

In [92]:
test_x = pca.fit_transform(p)

ValueError: n_components=50 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

In [88]:
test_x.shape

(1, 1)

In [89]:
test_x

array([[44.08682634]])

#### Doing What I Did Before

Building a pipeline with how I processed songs before. Here's what I need:

- Record key changes for each song section
- One-Hot Encoded Genres 
  - Then apply SVD
  - This should be easy; There shouldn't be any new artists, so I should be able to apply what I've got on a song level
- Gathering Mean & Variance of Audio Analysis Values